In [1]:
import selenium
import re
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By


In [10]:
class webscraper:
    def __init__(self, url="https://www.hltv.org/results"):
        self.url = url

    def open_results(self):
        browser = webdriver.Firefox()

        print("open website...")
        browser.get(self.url)
        print("get data...")
        game_links = browser.find_elements(By.CLASS_NAME, "a-reset")
        match_links = {}
        for e in game_links:
            link = e.get_attribute("href")
            s = link.split(sep = '/')
            if "matches" == s[3]:
                id = s[4]
                match_links[id] = link
        print("close website...")
        browser.close()
        return match_links

    def open_match(self,match_url):
        options = Options()
        options.add_argument("--headless")
        driver = webdriver.Firefox(options=options)
        
        driver.get(match_url)
        p_element = driver.find_elements(By.CLASS_NAME,'stats-menu-link')
        maps = []
        for e in p_element:
            maps.append(e.get_attribute('innerHTML'))

        p_element = driver.find_elements(By.CLASS_NAME,'stats-content')
        scores = []
        for e in p_element:
            scores.append(e.get_attribute("innerHTML"))
        driver.quit()

        return zip(maps, scores)
    

    def get_stats(self,data):
        game_stats = {}
        for map, players in data:
            pattern = r'id="[^"]*">([^"]*)</'
            map_match = re.findall(pattern, map)
            print(map_match[0])


            players = players.split('<td class="players">')

            player_dict = {}
            player_count = {}
            
            counts = {1:'total',
                    2:'ct',
                    3:'t'}

            for player in players:
                start = player.find("/player/")
                if start == -1:
                    continue
                end = start + player[start:].find('"')
                _,_,p_id, name = player[start:end].split('/')

                p_id = int(p_id)
                if p_id not in player_count.keys():

                    player_dict[p_id] = {'name':name}
                    player_count[p_id] = 1

                pattern = r'class="([^"]*((kd|adr|roundSwing|kast|rating)[^"]*(traditional-data|eco-adjusted-data|text-center)[^"]*)[^"]*)">([+-]?[\d\.\-%]+)</td>'
                matches = re.findall(pattern, player)
                stats = {}
                for match in matches:

                    classes = match[2]
                    value = match[4]
                    value = value.replace('%','')

                    if 'eco' in match[3]:
                        classes = 'e'+classes
                        
                    if "kd" in classes and "eco" in match[3]:
                        k,d = value.split('-')
                        stats['ek'] = float(k)
                        stats['ed'] = float(d)

                    elif "kd" in classes and "eco" not in match[3]:
                        k,d = value.split('-')
                        stats['k'] = float(k)
                        stats['d'] = float(d)
                    else:
                        stats[classes] = float(value)


                player_dict[p_id][counts[player_count[p_id]]] = stats

                player_count[p_id] += 1
            
            game_stats[map_match[0]]= player_dict

        return game_stats



In [14]:
scraper = webscraper(url = "https://www.hltv.org/results?stars=2")

matches = scraper.open_results()

open website...
get data...
close website...


In [15]:
scraper = webscraper()

for id, url in matches.items():
    data = scraper.open_match(url)
    stats = scraper.get_stats(data)
    print(stats)
    break

All
nuke
d2
inf
ovp
{'All': {18462: {'name': 'mezii', 'total': {'k': 54.0, 'd': 33.0, 'ek': 54.0, 'ed': 33.0, 'roundSwing': 5.26, 'adr': 73.0, 'eadr': 71.6, 'kast': 79.2, 'ekast': 78.9, 'rating': 1.41}, 'ct': {'k': 28.0, 'd': 14.0, 'ek': 30.0, 'ed': 15.0, 'roundSwing': 1.59, 'adr': 79.4, 'eadr': 82.9, 'kast': 77.4, 'ekast': 74.2, 'rating': 1.35}, 't': {'k': 26.0, 'd': 19.0, 'ek': 24.0, 'ed': 18.0, 'roundSwing': 8.03, 'adr': 68.2, 'eadr': 63.0, 'kast': 80.5, 'ekast': 82.5, 'rating': 1.46}}, 11816: {'name': 'ropz', 'total': {'k': 60.0, 'd': 46.0, 'ek': 59.0, 'ed': 51.0, 'roundSwing': 4.19, 'adr': 92.4, 'eadr': 90.5, 'kast': 75.0, 'ekast': 74.4, 'rating': 1.39}, 'ct': {'k': 27.0, 'd': 20.0, 'ek': 27.0, 'ed': 24.0, 'roundSwing': 2.81, 'adr': 98.5, 'eadr': 96.8, 'kast': 74.2, 'ekast': 74.1, 'rating': 1.31}, 't': {'k': 33.0, 'd': 26.0, 'ek': 32.0, 'ed': 28.0, 'roundSwing': 5.23, 'adr': 87.7, 'eadr': 85.7, 'kast': 75.6, 'ekast': 74.6, 'rating': 1.45}}, 11893: {'name': 'zywoo', 'total': {'k': 

In [16]:
import pprint

pprint.pp(stats)

{'All': {18462: {'name': 'mezii',
                 'total': {'k': 54.0,
                           'd': 33.0,
                           'ek': 54.0,
                           'ed': 33.0,
                           'roundSwing': 5.26,
                           'adr': 73.0,
                           'eadr': 71.6,
                           'kast': 79.2,
                           'ekast': 78.9,
                           'rating': 1.41},
                 'ct': {'k': 28.0,
                        'd': 14.0,
                        'ek': 30.0,
                        'ed': 15.0,
                        'roundSwing': 1.59,
                        'adr': 79.4,
                        'eadr': 82.9,
                        'kast': 77.4,
                        'ekast': 74.2,
                        'rating': 1.35},
                 't': {'k': 26.0,
                       'd': 19.0,
                       'ek': 24.0,
                       'ed': 18.0,
                       'roundSwing': 8.0